## Notebook de Analisis y Limpieza de Datos
Este notebook tiene como objetivo limpiar y preprocesar un conjunto de datos meteorológicos que incluye columnas numéricas y categóricas. 

Se aplicarán varios pasos de limpieza, como el manejo de valores nulos, la interpolación de valores faltantes, la detección de outliers, y la conversión de tipos de datos. 

Finalmente, se exportará el conjunto de datos limpio y listo el modelado de machine learning.


### Carga del dataset

In [73]:
import pandas as pd

# Cargar el dataset procesado
df = pd.read_csv('../data/processed/datos_procesados.csv')

# Verificar las primeras filas del dataset y los tipos de datos
df.head(), df.dtypes, df.describe()

(    Año Mes Día Temp Media Temp Máxima Hora Temp Max Minuto Temp Max  \
 0  2016   1   1       10.3        11.2            20              40   
 1  2016   1   2       11.8        18.1            19              30   
 2  2016   1   3       11.3        16.4            15               0   
 3  2016   1   4       12.5        18.6            15              20   
 4  2016   1   5       12.0        18.8            15              15   
 
   Temp Mínima Hora Temp Min Minuto Temp Min  ... Hora Viento Minuto Viento  \
 0         9.2             1              30  ...           6            55   
 1         8.6             0              55  ...          15            20   
 2         5.6             5              40  ...          10            35   
 3         6.1             2              50  ...          17            30   
 4         6.1            23              55  ...          16            15   
 
   Presión Media Lluvia Diaria Radiación Máxima Hora Radiación Máxima  \
 0        1

### 2. Verificar si hay Valores Faltantes
   
>Verificamos si hay valores faltantes en las columnas del dataset.

In [36]:
# Verificar si hay valores faltantes
df.isnull().sum()

Año                          0
Mes                         56
Día                         57
Temp Media                  57
Temp Máxima                 63
Hora Temp Max              231
Minuto Temp Max            231
Temp Mínima                231
Hora Temp Min              343
Minuto Temp Min            658
Viento Medio               658
Dirección Viento           868
Ráfaga de Viento           868
Dirección de Ráfaga        868
Hora Viento                926
Minuto Viento              926
Presión Media              982
Lluvia Diaria              982
Radiación Máxima           982
Hora Radiación Máxima      986
Minuto Radiación Mínima    986
UV Máximo                  986
Hora UV Máximo             986
Minuto UV Máximo           986
dtype: int64

### 3. Definir Columnas por Tipo de Dato
>Definimos las columnas por tipo de dato (entero, flotante, y categórico) para realizar las conversiones necesarias.

In [37]:
# Listas de columnas por tipo de dato requerido
columnas_int = [
    'Año', 'Mes', 'Día', 'Hora Temp Max', 'Minuto Temp Max', 
    'Hora Temp Min', 'Minuto Temp Min', 'Hora Viento', 'Minuto Viento', 
    'Hora Radiación Máxima', 'Minuto Radiación Mínima', 'Hora UV Máximo', 'Minuto UV Máximo'
]

columnas_float = [
    'Temp Media', 'Temp Máxima', 'Temp Mínima', 'Viento Medio', 'Ráfaga de Viento', 
    'Presión Media', 'Lluvia Diaria', 'Radiación Máxima', 'UV Máximo'
]

columnas_categorical = ['Dirección Viento', 'Dirección de Ráfaga']



### 4. Convertir Columnas a Tipos Numéricos
>Convertimos las columnas de tipo flotante y entero a su tipo correspondiente, forzando a NaN aquellos valores no convertibles.

In [38]:
# Convertir las columnas a float, forzando valores no convertibles a NaN
for col in columnas_float:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convertir las columnas a int, forzando valores no convertibles a NaN y redondeando si es necesario
for col in columnas_int:
    df[col] = pd.to_numeric(df[col], errors='coerce').round()


### 5. Rellenar Valores Nulos con Interpolación
>Llenamos los valores nulos en las columnas numéricas mediante interpolación lineal.

In [47]:
# Rellenar valores numéricos nulos mediante interpolación
df[columnas_float + columnas_int] = df[columnas_float + columnas_int].interpolate(method='linear')

# Convertir las columnas a Int64 después de la interpolación y el redondeo
for col in columnas_int:
    df[col] = df[col].round().astype('Int64')
    
# Rellenar valores nulos restantes en columnas categóricas (ejemplo con moda)
df['Dirección Viento'] = df['Dirección Viento'].fillna(df['Dirección Viento'].mode()[0])
df['Dirección de Ráfaga'] = df['Dirección de Ráfaga'].fillna(df['Dirección de Ráfaga'].mode()[0])

### 6. Convertir Columnas Categóricas
>Convertimos las columnas categóricas a tipo category.

In [48]:

# Convertir las columnas categóricas
for col in columnas_categorical:
    df[col] = df[col].astype('category')

### 7. Rellenar Valores Nulos en Columnas Categóricas con la Moda
>Rellenamos los valores nulos en las columnas categóricas utilizando la moda de cada columna.

In [49]:
# Opcional: Rellenar valores nulos restantes en columnas categóricas (ejemplo con moda)
for col in columnas_categorical:
    df[col] = df[col].fillna(df[col].mode()[0])

### 8. Filtrar el Dataset por Año
>Filtramos las filas donde la columna 'Año' esté entre 2016 y 2024, inclusive.

In [63]:
df = df[df['Año'].between(2016, 2024)]

# Eliminar filas donde 'Mes' o 'Día' tienen valores negativos
df = df[(df['Mes'] >= 0) & (df['Día'] >= 0)]


### 9. Detectar y Corregir Outliers
>Usamos el método del rango intercuartílico (IQR) para detectar y corregir los outliers en las columnas numéricas.

In [64]:
# Función para detectar outliers usando el IQR
def detect_outliers(df, columnas):
    for col in columnas:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        # Definir los límites superior e inferior para los outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Reemplazar los outliers por los valores dentro del rango permitido
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
    return df

# Listado de columnas a revisar (en este caso numéricas)
columnas_a_verificar = [
    'Temp Media', 'Temp Máxima', 'Temp Mínima', 'Viento Medio', 'Ráfaga de Viento', 
    'Presión Media', 'Lluvia Diaria', 'Radiación Máxima', 'UV Máximo'
]

# Llamamos a la función para detectar y corregir outliers
df = detect_outliers(df, columnas_a_verificar)



### 10. Verificar Resultados Finales
>Verificamos los resultados después de la limpieza y preprocesamiento.

In [65]:
# Verifica los resultados
df.describe()

# Verificar valores nulos nuevamente
df.isnull().sum()


Año                        0
Mes                        0
Día                        0
Temp Media                 0
Temp Máxima                0
Hora Temp Max              0
Minuto Temp Max            0
Temp Mínima                0
Hora Temp Min              0
Minuto Temp Min            0
Viento Medio               0
Dirección Viento           0
Ráfaga de Viento           0
Dirección de Ráfaga        0
Hora Viento                0
Minuto Viento              0
Presión Media              0
Lluvia Diaria              0
Radiación Máxima           0
Hora Radiación Máxima      0
Minuto Radiación Mínima    0
UV Máximo                  0
Hora UV Máximo             0
Minuto UV Máximo           0
dtype: int64

In [69]:
# Exportar el dataframe limpio y procesado
df.to_csv('../data/processed/datos_limpios.csv', index=False)

In [72]:
df.head()

,Año,Mes,Día,Temp Media,Temp Máxima,Hora Temp Max,Minuto Temp Max,Temp Mínima,Hora Temp Min,Minuto Temp Min,...,Hora Viento,Minuto Viento,Presión Media,Lluvia Diaria,Radiación Máxima,Hora Radiación Máxima,Minuto Radiación Mínima,UV Máximo,Hora UV Máximo,Minuto UV Máximo
0,2016,1,1,10.3,11.2,20,40,9.2,1,30,...,6,55,1004.6,0.0,256.0,12,25,1.5,12,25
1,2016,1,2,11.8,18.1,19,30,8.6,0,55,...,15,20,996.2,0.2,798.0,13,20,4.9,13,25
2,2016,1,3,11.3,16.4,15,0,5.6,5,40,...,10,35,1004.3,0.0,1186.0,15,0,5.1,13,50
3,2016,1,4,12.5,18.6,15,20,6.1,2,50,...,17,30,1004.1,0.0,1096.0,12,55,5.0,12,55
4,2016,1,5,12.0,18.8,15,15,6.1,23,55,...,16,15,1005.4,0.0,1055.0,15,10,4.3,15,15


## Visualizacion de los datos